In [32]:
import pandas as pd
import numpy as np
import os, sys
from datetime import date
import json

pd.set_option("display.max_rows", 100, "display.max_columns", 100)

In [2]:
def p(s):
    print(s)
    
pd.DataFrame.len = pd.Index.len = lambda x: print(len(x))

In [3]:
if os.path.exists('data/data.csv.gz'):
    df = pd.read_csv('data/data.csv.gz', compression='gzip',index_col=0)
elif os.path.exists('../data/data.csv.gz'):
    df = pd.read_csv('../data/data.csv.gz', compression='gzip',index_col=0)
else:
    YEARS = np.arange(1999,date.today().year)
    df = pd.DataFrame()
    for i in YEARS:
        sys.stdout.write(f'{i} \r'); sys.stdout.flush()
        iData = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                             'play_by_play_' + str(i) + '.csv.gz?raw=True',
                             compression='gzip', low_memory=False)
        df = df.append(iData, sort=True)

    df.reset_index(drop=True, inplace=True)
    df.to_csv('../data/data.csv.gz', compression='gzip')

C:\Users\mrkawa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (11,12,13,14,15,16,23,24,62,63,79,80,81,93,94,95,100,101,102,132,133,134,135,136,137,138,139,144,145,150,157,158,164,165,166,167,184,204,205,206,207,219,249,250,251,255,257,260,261,262,263,306,310) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
d = df[df.season==2019].reset_index(drop=True)
if not os.path.exists('../data/2019.csv.gz'):
    d.to_csv('../data/2019.csv.gz', compression='gzip')

In [88]:
colSearch = 'return'
d[[col for col in d.columns if colSearch in col]].columns
# d[[col for col in d.columns if col.startswith(colSearch)]].head()

Index(['kickoff_returner_player_id', 'kickoff_returner_player_name',
       'lateral_kickoff_returner_player_id',
       'lateral_kickoff_returner_player_name',
       'lateral_punt_returner_player_id', 'lateral_punt_returner_player_name',
       'lateral_return', 'punt_returner_player_id',
       'punt_returner_player_name', 'return_team', 'return_touchdown',
       'return_yards'],
      dtype='object')

In [34]:
idCols = ['passer_id','rusher_id','receiver_id','kickoff_returner_player_id','punt_returner_player_id']
valCols = ['season','game_id','yards_gained','qb_dropback','shotgun','no_huddle','qb_kneel','qb_spike','qb_scramble','air_yards','yards_after_catch','first_down_rush','first_down_pass','third_down_converted','third_down_failed','fourth_down_converted','fourth_down_failed','incomplete_pass','interception','fumble_forced','fumble_not_forced','fumble_out_of_bounds','safety','penalty','tackled_for_loss','fumble_lost','qb_hit','rush_attempt','pass_attempt','sack','touchdown','pass_touchdown','rush_touchdown','return_touchdown','return_yards','fumble','complete_pass']
notUsed = [col for col in d.columns if col not in idCols + valCols]
df = df[idCols + valCols]

game = df.set_index(valCols).stack().reset_index().groupby(['season','game_id',0]).sum()
season = df.set_index(valCols).stack().reset_index().groupby(['season',0]).sum()

game.to_csv('../data/gameData.csv')
season.to_csv('../data/seasonData.csv')

In [82]:
colDict = eval(open('../data/columnDefinitions.txt', 'r').read())
colDefs = [(col,colDict[col]) for col in colDict.keys() if col in game.columns]
for k,v in colDefs:
    print(f'* {k}:\t {v}\n')

* yardline_100:	 Numeric distance in the number of yards from the opponent's endzone for the posteam.

* quarter_seconds_remaining:	 Numeric seconds remaining in the quarter.

* half_seconds_remaining:	 Numeric seconds remaining in the half.

* game_seconds_remaining:	 Numeric seconds remaining in the game.

* quarter_end:	 Binary indicator for whether or not the row of the data is marking the end of a quarter.

* drive:	 Numeric drive number in the game.

* sp:	 Binary indicator for whether or not a score occurred on the play.

* qtr:	 Quarter of the game (5 is overtime).

* down:	 The down for the given play.

* goal_to_go:	 Binary indicator for whether or not the posteam is in a goal down situation.

* ydstogo:	 Numeric yards in distance from either the first down marker or the endzone in goal down situations.

* ydsnet:	 Numeric value for total yards gained on the given drive.

* yards_gained:	 Numeric yards gained (or lost) for the given play.

* shotgun:	 Binary indicator for whe